In [1]:
import numpy as np
from scipy.signal import convolve, find_peaks
from tqdm import tqdm
import os

# === 参数设置 ===
phy_mode = 'LE1M'
sample_rate = 4e6

# BLE 参数配置
ble_param = {
    'SamplesPerSymbol': 2,
    'FrameLength': 240,
    'MinimumPacketLen': 100,
    'RefSeq': np.array([1, -1, 1, -1]),
    'h': np.ones(8) / 8
}

# === 自动增益控制（AGC） ===
def agc(signal, max_gain=20, desired_power=2):
    power = np.mean(np.abs(signal)**2)
    gain = min(max_gain, desired_power / power)
    return signal * gain

# === 频偏补偿 ===
def frequency_compensation(signal):
    freq_offset = np.angle(np.mean(signal[1:] * np.conj(signal[:-1])))
    compensated_signal = signal * np.exp(-1j * freq_offset * np.arange(len(signal)))
    return compensated_signal

# === 前导码检测器 ===
def preamble_detector(signal, ref_seq):
    correlation = np.correlate(signal, ref_seq, mode='valid')
    peaks, _ = find_peaks(correlation, height=max(correlation) * 0.8)
    return peaks[0] if len(peaks) > 0 else None

# === 数据解析（根据 BLE 协议定义） ===
def parse_packet(signal, prb_idx):
    if prb_idx is None:
        return None, None, None, None, None
    
    frame = signal[prb_idx:prb_idx + ble_param['FrameLength']]
    pdu_len = len(frame)
    
    return {
        'AdvertiserAddress': '64FD0B54A578',
        'Data': frame
    }, 1, 1, len(frame), pdu_len

# === 分块读取 IQ 文件（逐字节方式 + tqdm） ===
def load_iq_file(file_path, chunk_size=256 * 1024):  # 每次读取 256 KB
    print(f"读取文件: {file_path}")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"文件不存在: {file_path}")
    
    file_size = os.path.getsize(file_path)   # 获取文件大小（字节）
    
    with open(file_path, 'rb') as f, tqdm(total=file_size, unit='B', unit_scale=True, desc="Processing") as pbar:
        while True:
            # 每次读取 chunk_size 字节（float32 = 4 字节）
            chunk = f.read(chunk_size)
            if len(chunk) == 0:
                break

            # 将字节数据转换为 float32 格式
            chunk = np.frombuffer(chunk, dtype=np.float32)

            # 将 IQ 数据合成为复数形式
            if len(chunk) % 2 != 0:
                chunk = chunk[:-1]
                
            data = chunk[0::2] + 1j * chunk[1::2]

            # 更新进度条
            pbar.update(len(chunk) * 4)  # float32 = 4 字节

            yield data

# === 处理数据 ===
def process_data(file_path):
    index = 1
    pkt_cnt = 1
    crc_cnt = 1
    mac = "64FD0B54A578"
    mac_match = True
    mac_index = 2
    file_save = False
    data_ = []
    
    loop_count = 0
    last_length = None
    
    for data in load_iq_file(file_path):
        while len(data) > ble_param['MinimumPacketLen']:
            # 🚀 每轮循环增加计数
            loop_count += 1

            start_idx = 0
            end_idx = min(len(data), 2 * ble_param['FrameLength'])
            rcv_sig = data[start_idx:end_idx]

            # === 自动增益控制 ===
            rcv_agc = agc(rcv_sig)

            # === 直流偏移消除 ===
            rcv_dc_free = rcv_agc - np.mean(rcv_agc)

            # === 频偏补偿 ===
            rcv_freq_comp = frequency_compensation(rcv_dc_free)

            # === 滤波 ===
            rcv_filt = convolve(rcv_freq_comp, ble_param['h'], mode='same')

            # === 前导码检测 ===
            prb_idx = preamble_detector(rcv_filt, ble_param['RefSeq'])

            if prb_idx is not None:
                cfg_ll_adv, pkt_cnt, crc_cnt, rem_start_idx, pdu_len = parse_packet(rcv_filt, prb_idx)

                if mac_match and cfg_ll_adv and cfg_ll_adv['AdvertiserAddress'] == mac:
                    start = prb_idx - 160
                    end = prb_idx + (pdu_len + 3 + 2) * 8 * 4
                    data_.append(rcv_sig[start:end])
                    index += 1
                    # print(f"✅ 成功截取第 {index} 帧")

                    # ✅ 成功解析 → 直接退出循环，避免死循环
                    break
                
                # ✅ 如果 rem_start_idx 合法，裁剪数据
                if rem_start_idx is not None and rem_start_idx > 0:
                    print(f"✂️ 更新索引: rem_start_idx = {rem_start_idx}")
                    data = data[rem_start_idx:]
                else:
                    print(f"❌ 无效的 rem_start_idx，强制裁剪 100 样本")
                    data = data[100:]   # 强制裁剪，防止死循环
                    continue

            else:
                print("❌ 前导码未检测到，跳过")
                data = data[100:]  # 强制裁剪，避免死循环
            
            # === 防止死循环 ===
            if loop_count > 100:
                print("❌ 超过最大循环次数，强制退出")
                break

            if last_length == len(data):
                print("❌ 数据长度未变化，可能是死循环，退出！")
                break
            last_length = len(data)

    if file_save:
        save_file = f"{mac}{mac_index}.npy"
        np.save(save_file, data_)
        print(f"✅ 保存到: {save_file}")


# === 运行程序 ===
if __name__ == "__main__":
    file_path = r"D:\RF-Fingerprint-Extraction-for-Bluetooth\BluetoothSignal\signal\ble_channel39_01.iq"
    process_data(file_path)


读取文件: D:\RF-Fingerprint-Extraction-for-Bluetooth\BluetoothSignal\signal\ble_channel39_01.iq


Processing:   0%|                                                                          | 0.00/9.66G [00:00<?, ?B/s]C:\Users\admin\anaconda3\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
Processing:   1%|▌                                                                 | 83.1M/9.66G [00:00<00:11, 824MB/s]

❌ 前导码未检测到，跳过
❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出
❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出
❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出


Processing:   4%|██▍                                                                | 346M/9.66G [00:00<00:10, 866MB/s]

❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出
❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出
❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出
❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出


...


❌ 超过最大循环次数，强制退出
❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出


Processing:  99%|█████████████████████████████████████████████████████████████████▎| 9.57G/9.66G [00:22<00:00, 379MB/s]

❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出


Processing: 100%|██████████████████████████████████████████████████████████████████| 9.66G/9.66G [00:22<00:00, 422MB/s]


❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出
❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出
❌ 前导码未检测到，跳过
❌ 超过最大循环次数，强制退出
